In [1]:
from dataclasses import dataclass
from sparkai.llm.llm import ChatSparkLLM, ChunkPrintHandler
from sparkai.core.messages import ChatMessage
import pandas as pd
import os
import json
import re
import matplotlib.pyplot as plt
from tqdm import tqdm
from math import ceil
import numpy as np
from copy import deepcopy
import random

In [2]:
tqdm.pandas()
plt.rcParams['font.family'] = ['STFangsong']
plt.rcParams['axes.unicode_minus'] = False


data_dir = "./dataset"
train_file = "train.json"
test_file = "test_data.json"

In [3]:
train_data = pd.read_json(os.path.join(data_dir, train_file))

test_data =  pd.read_json(os.path.join(data_dir, test_file))

In [4]:
# 删除表情图片、超链接
train_data['chat_text'] = train_data['chat_text'].str.replace(r"\[[^\[\]]{2,10}\]", "", regex=True)
train_data['chat_text'] = train_data['chat_text'].str.replace("https?://\S+", "", regex=True)
test_data['chat_text'] = test_data['chat_text'].str.replace(r"\[[^\[\]]{2,10}\]", "", regex=True)
test_data['chat_text'] = test_data['chat_text'].str.replace("https?://\S+", "", regex=True)

In [5]:
def get_names_phones_and_emails(example):
    names = re.findall(r"(?:\n)?([\u4e00-\u9fa5]+\d+)：", example["chat_text"])
    names += re.findall(r"@([\u4e00-\u9fa5]+)\s", example["chat_text"])
    emails = re.findall(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}", example["chat_text"])
    # phones = re.findall(r"1[356789]\d{9}", example["chat_text"]) # 文本中的手机号并不是标准手机号
    phones = re.findall(r"\d{3}\s*\d{4}\s*\d{4}", example["chat_text"])
    return pd.Series([set(names), set(phones), set(emails)], index=['names', 'phones', 'emails'])

In [6]:
def merge_chat(example):
    for name in example['names']:
        example["chat_text"] = example["chat_text"].replace(f"\n{name}：", f"<|sep|>{name}：")
    chats = example["chat_text"].split("<|sep|>")

    last_name = "UNKNOWN"
    new_chats = []
    for chat in chats:
        if chat.startswith(last_name):
            chat = chat.strip("\n")
            chat = "".join(chat.split("：")[1:])
            new_chats[-1] += " " + chat
        else:
            new_chats.append(chat)
            last_name = chat.split("：")[0]
    return pd.Series(["\n".join(new_chats), new_chats], index=["chats", "chat_list"])

In [7]:
# 使用正则表达式获得'names', 'phones', 'emails'
train_data[['names', 'phones', 'emails']] = train_data.apply(get_names_phones_and_emails, axis=1)
test_data[['names', 'phones', 'emails']] = test_data.apply(get_names_phones_and_emails, axis=1)
test_data

,chat_text,infos,names,phones,emails
0,周强7：获客助手申请链接发一下\n王艳9：\n钱艳4：咱们巨石蓝海可以通过api接入吗？\n...,[],"{周娜4, 吴强8, 周强7, 鸡蛋, 王艳9, 周杰5, 吴军7, 浮尘, 钱艳4, 郑芳...",{16795386856},{}
1,赵军2：\n赵军2：我们这边准备在1月完成两个功能的续费，麻烦帮忙计算一下所有涉及到的经费的...,[],"{王勇7, 赵军2}",{10404514580},{}
2,王勇7：李强总 手办到了 我下午给你带过去\n王勇7：大概2点前到\n王勇7：你在公司嘛\n...,[],"{李强5, 王勇7}",{},{}
3,赵伟7：咱们小程序出单，能出这个流水号不\n王艳9：不行的哈\n吴娜8：\n吴娜8：客户到店...,[],"{李娜2, 赵伟7, 孙伟3, 周杰5, 赵伟3, 弥陀, 微梦, 郑杰1, 孙杰3, 孙娜...",{},{}
4,周杰5：客户到店频次提升50%，驰加是如何与巨石蓝海一起基于企业微信做增长的？\n周杰5：\...,[],"{赵艳9, 钱伟8, 李艳2, 周杰5, 王杰9, 周伟1, 钱磊1, 李洋4, 钱伟5, ...",{},{}
5,王伟5：\n王伟5：@王勇3 这个号为什么，全是显示未知联系人\n王艳9：您的会话存档是什么...,[],"{潘亚伟, 周杰5, 王伟5, 王勇3, 孙磊9, 周洋8, 苗苗, 王艳9}",{},{}
6,赵艳6：@陆由 巨石蓝海·企微管家 登陆不了账号是什么原因呢\n王杰4：@吴伟9 巨石蓝海·...,[],"{吴军6, 李芳1, 钱勇3, 赵芳3, 王艳9, 李娜5, 王杰4, 李杰5, 陆由, 吴...",{},{}
7,王勇4：周磊1总，咱们沐心的系统12月29日就到期了，为了不影响您们使用，您看我是最近把合同...,[],"{周磊1, 王勇4}",{},{}
8,王勇7：腾飞集团是要续约嘛\n周芳1：嗯，\n周芳1：我们小程序好像到期了，打不开\n周芳1...,[],"{周芳1, 王勇7}",{11333928977},{}
9,周勇8：\n周勇8：\n王勇7：周勇8您方便电话沟通下吗\n周勇8：好的\n周勇8：1322...,[],"{周勇8, 郑军7, 赵伟9, 王勇7}",{13221352438},{}


In [8]:
# 分割聊天记录， 合并连续相同人的聊天
train_data[["chats", "chat_list"]] = train_data.apply(merge_chat, axis=1)
test_data[["chats", "chat_list"]] = test_data.apply(merge_chat, axis=1)

In [9]:
#数据中chat_text中有许多是重复多编的，我们需要把重复的也给去除掉，这样处理后的数据就会大大减小，使用暴力匹配去除
def process(excemple):
    chat_list = excemple["chat_text"].split("\n")

    res = []
    s = 0
    while s < len(chat_list):

        i, j = s, s + 1
        start_j = j
        while i < len(chat_list) and j < len(chat_list):
            if chat_list[i] == chat_list[j]:
                i += 1
            else:
                if i != s:
                    if j - start_j > 10:
                        res += list(range(start_j, j))
                    i = s
                start_j = j
            j += 1
        s += 1
    texts = []
    for i in range(len(chat_list)):
        if i not in res:
            texts.append(chat_list[i])
    return "\n".join(texts)

In [10]:
train_data["chat_text"] = train_data.apply(process, axis=1)
print(len(train_data))
test_data["chat_text"] = test_data.apply(process, axis=1)
print(train_data)

129
                                             chat_text  \
0    李勇7：\n王艳9：您指的什么，辛苦截图告知下\n王艳9：您是企业微信管理员吗？\n王艳9：...   
1    赵杰5：我是巨石蓝海企微管家的赵杰5\n孙洋2：我通过了你的联系人验证请求，现在我们可以开始...   
2    王艳9：用标签来区分的哈\n吴伟7：可以设置一部分标签客户不掉公海吗\n王勇3：您好，在公海...   
3    郑杰1：客户到店频次提升50%，驰加是如何与巨石蓝海一起基于企业微信做增长的？\n赵艳9：各...   
4    赵艳2：哈喽\n赵艳2：你们现在还是再做企微运营这块么\n王勇7：在做的\n赵艳2：下午两点...   
..                                                 ...   
124  李强8：\n李强8：\n李强8：\n王勇7：\n王勇7：\n王勇7：\n王勇7：\n王勇7：...   
125  李娜9：客户到店频次提升50%，驰加是如何与巨石蓝海一起基于企业微信做增长的？\n李娜9：\...   
126  郑杰1：\n赵洋2：@郑杰1（巨石蓝海 企微管家） \n王艳9：\n王艳9：你好，客户跟进可...   
127  王磊8：@郑杰1（巨石蓝海 企微管家） \n郑杰1：自建应用要安装配置的\n郑杰1：没有配置...   
128  李强7：我通过了你的联系人验证请求，现在我们可以开始聊天了\n李强7：\n李强7：微信电话吗...   

                                                 infos  \
0    [{'基本信息-姓名': '动漫周边配送中心', '基本信息-手机号码': '', '基本信...   
1    [{'基本信息-姓名': '孙洋2', '基本信息-手机号码': '', '基本信息-邮箱'...   
2    [{'基本信息-姓名': '吴伟7', '基本信息-手机号码': '', '基本信息-邮箱'...   
3    [{'基本信息-姓名': '孙艳3', '基本信息-手机号码': '', '基本信息-邮箱'...   
4    [{'基

In [18]:
train_data.iloc[0]['chat_text']


'李勇7：\n王艳9：您指的什么，辛苦截图告知下\n王艳9：您是企业微信管理员吗？\n王艳9：管理员可以看到\n王艳9：\n王艳9：\n王艳9：那让管理员给您添加应用负责人\n王艳9：也可以看到\n王艳9：@动漫周边配送中心\n王艳9：电脑端不可以\n王艳9：要登录管理端后台\n王艳9：这是一条引用/回复消息：\n"王艳9\n\n------\n这个网址\n王艳9：\n郑杰1：@动漫周边配送中心 您登记应用管理——点击打卡 里面有的哈 \n郑杰1：\n郑杰1：使用企业微信完成20%的新客增量提升，老客返店率增长至50%！\n巨石蓝海如何助力燕京啤酒旗下小酒馆实现品牌新增长？\n郑杰1：????上线功能：H5红包改版\n需求背景：当前的H5红包功能较弱，只支持创建一种金额的红包给员工发放，对需要给不同门店、不同业务线的员工创建不同的红包活动的场景，就不太好满足\n\n具体上新功能点介绍：\n①支持创建客户红包和客户群红包，员工在聊天工具栏可一键发送红包，用于1v1客户运营和客户群促活场景\n②可创建指定金额或员工自定义金额的红包，用于对客户的分层运营\n③支持转发红包给他人后不允许领取，防止被薅羊毛\n\n注：旧红包工具将在12月31日下线，请注意切换到新的红包功能使用 \n郑杰1：您是指要看到什么内容呢 \n郑杰1：看不到这个新客户页面是吗 \n郑杰1：\n郑杰1：您的企业微信更新到最新版本之后 退出 重新登录再试试呢 \n郑杰1：电脑端的话 你点击你的头像 点击设置 点击关于 \n郑杰1：检查版本 更新\n郑杰1：\n郑杰1：你点击关于里面\n郑杰1：去更新版本之后再看看 \n周杰5：学员、订单双增长的背后，EF是如何用好企业微信和企微管家优化服务链路，让“短期教学”变成“终身教育”？\n周杰5：\n周杰5：各位老师好，新的一年已经开启，我们诚邀您对我们2023年的服务和产品做出评价。辛苦在表格内反馈您的宝贵建议，您的反馈会帮助我们做的更好，感谢您的支持！\n【收集表】 2023年度服务满意度评价\xa0 \n王艳9：\n王艳9：这里看下呢\n王艳9：点头像\n王艳9：然后最下面有个设置\n王艳9：重启试下把'

In [20]:
train_data.iloc[0]['names']

{'动漫周边配送中心',
 '周勇3',
 '周杰5',
 '周洋8',
 '孙勇7',
 '李勇7',
 '李强5',
 '王磊4',
 '王艳9',
 '赵磊5',
 '郑娜3',
 '郑杰1'}

In [21]:
train_data.iloc[0]['chats']

'李勇7：\n王艳9：您指的什么，辛苦截图告知下 您是企业微信管理员吗？ 管理员可以看到   那让管理员给您添加应用负责人 也可以看到 @动漫周边配送中心 电脑端不可以 要登录管理端后台 这是一条引用/回复消息\n"王艳9\n\n------\n这个网址 \n郑杰1：@动漫周边配送中心 您登记应用管理——点击打卡 里面有的哈   使用企业微信完成20%的新客增量提升，老客返店率增长至50%！\n巨石蓝海如何助力燕京啤酒旗下小酒馆实现品牌新增长？ ????上线功能H5红包改版\n需求背景当前的H5红包功能较弱，只支持创建一种金额的红包给员工发放，对需要给不同门店、不同业务线的员工创建不同的红包活动的场景，就不太好满足\n\n具体上新功能点介绍\n①支持创建客户红包和客户群红包，员工在聊天工具栏可一键发送红包，用于1v1客户运营和客户群促活场景\n②可创建指定金额或员工自定义金额的红包，用于对客户的分层运营\n③支持转发红包给他人后不允许领取，防止被薅羊毛\n\n注旧红包工具将在12月31日下线，请注意切换到新的红包功能使用  您是指要看到什么内容呢  看不到这个新客户页面是吗   您的企业微信更新到最新版本之后 退出 重新登录再试试呢  电脑端的话 你点击你的头像 点击设置 点击关于  检查版本 更新  你点击关于里面 去更新版本之后再看看 \n周杰5：学员、订单双增长的背后，EF是如何用好企业微信和企微管家优化服务链路，让“短期教学”变成“终身教育”？  各位老师好，新的一年已经开启，我们诚邀您对我们2023年的服务和产品做出评价。辛苦在表格内反馈您的宝贵建议，您的反馈会帮助我们做的更好，感谢您的支持！\n【收集表】 2023年度服务满意度评价\xa0 \n王艳9： 这里看下呢 点头像 然后最下面有个设置 重启试下把" data-online-sheet-link=\'["李勇7客户到店频次提升50%，驰加是如何与巨石蓝海一起基于企业微信做增长的？\\周勇3\\孙勇7您指的什么，辛苦截图告知下\\孙勇7您是企业微信管理员吗？\\孙勇7管理员可以看到\\孙勇7\\孙勇7 您登记应用管理——点击打卡 里面有的哈 \\王磊4\\王磊4使用企业微信完成20%的新客增量提升，老客返店率增长至50%！\\n巨石蓝海如何助力燕京啤酒旗下小酒馆实现品牌新增长？\\王磊4????上线

In [22]:
train_data.iloc[0]['chat_list']

['李勇7：',
 '王艳9：您指的什么，辛苦截图告知下 您是企业微信管理员吗？ 管理员可以看到   那让管理员给您添加应用负责人 也可以看到 @动漫周边配送中心 电脑端不可以 要登录管理端后台 这是一条引用/回复消息\n"王艳9\n\n------\n这个网址 ',
 '郑杰1：@动漫周边配送中心 您登记应用管理——点击打卡 里面有的哈   使用企业微信完成20%的新客增量提升，老客返店率增长至50%！\n巨石蓝海如何助力燕京啤酒旗下小酒馆实现品牌新增长？ ????上线功能H5红包改版\n需求背景当前的H5红包功能较弱，只支持创建一种金额的红包给员工发放，对需要给不同门店、不同业务线的员工创建不同的红包活动的场景，就不太好满足\n\n具体上新功能点介绍\n①支持创建客户红包和客户群红包，员工在聊天工具栏可一键发送红包，用于1v1客户运营和客户群促活场景\n②可创建指定金额或员工自定义金额的红包，用于对客户的分层运营\n③支持转发红包给他人后不允许领取，防止被薅羊毛\n\n注旧红包工具将在12月31日下线，请注意切换到新的红包功能使用  您是指要看到什么内容呢  看不到这个新客户页面是吗   您的企业微信更新到最新版本之后 退出 重新登录再试试呢  电脑端的话 你点击你的头像 点击设置 点击关于  检查版本 更新  你点击关于里面 去更新版本之后再看看 ',
 '周杰5：学员、订单双增长的背后，EF是如何用好企业微信和企微管家优化服务链路，让“短期教学”变成“终身教育”？  各位老师好，新的一年已经开启，我们诚邀您对我们2023年的服务和产品做出评价。辛苦在表格内反馈您的宝贵建议，您的反馈会帮助我们做的更好，感谢您的支持！\n【收集表】 2023年度服务满意度评价\xa0 ',
 '王艳9： 这里看下呢 点头像 然后最下面有个设置 重启试下把" data-online-sheet-link=\'["李勇7客户到店频次提升50%，驰加是如何与巨石蓝海一起基于企业微信做增长的？\\周勇3\\孙勇7您指的什么，辛苦截图告知下\\孙勇7您是企业微信管理员吗？\\孙勇7管理员可以看到\\孙勇7\\孙勇7 您登记应用管理——点击打卡 里面有的哈 \\王磊4\\王磊4使用企业微信完成20%的新客增量提升，老客返店率增长至50%！\\n巨石蓝海如何助力燕京啤酒旗下小酒馆实现品牌新增

In [20]:
spark = ChatSparkLLM(
    spark_api_url="wss://spark-api.xf-yun.com/v4.0/chat",
    spark_app_id="403f6768",
    spark_api_key="bb55d4b872e8a0fd5ab8dab00c676598",
    spark_api_secret="ZjEyNTIwZWFkYWQ5M2U4MjdmOTk2MzMx",
    spark_llm_domain="4.0Ultra", #spark 4.0
    temperature = 0.00001,
    max_tokens = 8192,
    request_timeout = 60,
    streaming=False,
)

In [22]:
#去除无关的广告等
def remove_repeat_model(data):

    chat_text = data["chat_text"]

    prompt_clean = """请对以下客服和客户之间的聊天记录进行数据清洗，并提取客户的重要信息。你的任务是提取并保留对话中的重要信息，剔除垃圾数据和重复内容。请确保每一条保留的信息都清晰、简洁，并能够反映对话的核心内容。

请对以下数据进行数据清洗：
```
{content}
```

以下是具体要求：
1. **保留的内容**：
    - 客服和客户的发言。
    - 每次对话的关键问题和回答。

2. **剔除的内容**：
    - 广告、无关链接、系统消息、自动回复等垃圾数据。
    - 任何形式的重复内容。
    - 空对话。

3. **合并同一人连续的发言**：
    - 如果同一个人在对话中连续发言，请将他们的发言内容合并为一条，使用空格隔开。

3. **内容格式**：
    - 每条对话内容按时间顺序排列。
    - 不同的人的发言使用“\n”进行隔开。

注意：请确保在你的清洗之后，你返回的内容是如下格式，除此之外不要回复其他任何信息。以下是你的回复格式：
```
{{"清洗结果":"结果内容"}}
```
"""
    retry = 8 # 报错时重新尝试
    for _ in range(retry):
        try:
            messages = [ChatMessage(role="user",content=chat_text)]
            handler = ChunkPrintHandler()
            res = spark.generate([messages], callbacks=[handler])
            #res = get_completions(prompt_clean.format(content=chat_text))
            response = res.replace('“','"').replace('”','"').replace('：',':').replace('，',',')
            response = response.replace('\n','<SEP>')
            json_str = re.search(r'\{.*\}', response).group()
            data = json.loads(json_str)
            return data["清洗结果"].replace(':','：').replace(',','，').replace('<SEP>','\n')
        except json.decoder.JSONDecodeError as json_error:
            print(json_error)
            print("- - - - - json_error- - - - - - - - -")
            print(res)
            print("- - - - - json_error- - - - - - - - -")
        except AttributeError as att_error:
            print(att_error)
            print("- - - - - - AttributeError- - - - - - - -")
            print(res)
            print("- - - - - - AttributeError- - - - - - - -")
        except Exception as e :
            print(e)
    return chat_text

In [23]:
train_data['new_chat_text'] = train_data.apply(remove_repeat_model, axis=1)

'LLMResult' object has no attribute 'replace'
- - - - - - AttributeError- - - - - - - -
generations=[[ChatGeneration(text='这段对话看起来像是来自一个企业微信的聊天群组，其中包含了多个用户之间的互动。主要内容涉及企业微信的应用管理、功能更新、客户运营策略以及服务满意度评价等方面。下面是对这段对话的简要分析：\n\n1. **应用管理和权限问题**：\n   - 王艳9询问李勇是否是企业微信管理员，并指出管理员可以看到某些信息。\n   - 王艳9建议让管理员添加应用负责人，以便能够查看相关信息。\n\n2. **功能更新和操作指南**：\n   - 郑杰1提到了企业微信的新功能，包括H5红包改版，支持创建不同类型的红包，用于不同的客户运营场景，并提醒旧红包工具将在12月31日下线。\n   - 郑杰1还提供了关于如何检查和更新企业微信版本的指导。\n\n3. **客户运营和服务优化**：\n   - 郑杰1提到了使用企业微信完成新客增量提升和老客返店率增长的案例。\n   - 周杰5讨论了EF如何使用企业微信和企微管家优化服务链路，将“短期教学”转变为“终身教育”。\n\n4. **服务满意度评价**：\n   - 周杰5邀请群组成员对2023年的服务和产品进行评价，并提供了一个收集表链接。\n\n5. **技术问题解决**：\n   - 王艳9指导如何通过重启应用来尝试解决看不到新客户页面的问题。\n\n整体来看，这段对话反映了企业微信作为企业内部沟通和客户运营管理工具的实际应用情况，包括功能更新、操作问题解决、客户运营策略分享以及服务反馈收集等多个方面。', message=AIMessage(content='这段对话看起来像是来自一个企业微信的聊天群组，其中包含了多个用户之间的互动。主要内容涉及企业微信的应用管理、功能更新、客户运营策略以及服务满意度评价等方面。下面是对这段对话的简要分析：\n\n1. **应用管理和权限问题**：\n   - 王艳9询问李勇是否是企业微信管理员，并指出管理员可以看到某些信息。\n   - 王艳9建议让管理员添加应用负责人，以便能够查看相关信息。\n\n2. **功能更新和操作指南**：\n   - 郑杰1

2024-07-16 15:27:54 中国标准时间 - SparkPythonSDK - ERROR - [E:\python_env\iflytek\lib\site-packages\sparkai\llm\llm.py:689] - SparkLLMClient wait LLM api response timeout 60 seconds


'LLMResult' object has no attribute 'replace'
- - - - - - AttributeError- - - - - - - -
generations=[[ChatGeneration(text='提供！咨询请加V:lijun1\n\n李军7：承接视频号--粉丝、视频播放量、点赞、转发、收藏、评论、直播在线人气(看过)，直播互动、等业务！稳定上量，有速度，咨询请加V: lijun1\n\n李军7：专业提供各类社交平台账号（新号、老号、带粉号都有）：脸书、油管、推特、ins、TikTok(可指定国家和指定粉丝数量)、chatGPT、Reddit、领英等平台，保首登，现货可批量提供！咨询请加V:lijun1\n\n李军7：Facebook/推特/ins/油管/tiktok/小红书 \n粉丝/点赞/评论/浏览量/转推等数据维护\n另出售各种白号、千粉、万粉账号！欢迎咨询：lijun2，可对公，可发票！\n\n李军7：承接 《小红书、视频号》--粉丝、评论、点赞、大拇指、评论点赞、小眼睛，看过等稳定上量，有速度，咨询请加V: lijun1 可对公，可发票！！\n\n李军7：【拥有资源】源头一手Tik*Tok号商！美、英、马来、印尼、越、泰、新、菲等多个国家都有！\n【联系方式】加 郑娜6：lijun2\n\n李军7：TikTok全类型账号一手出售，满月白号，老号，满月千粉，英区，美区五千粉，还有印尼，马来，菲律宾，泰国，越南，沙特等全球满月千粉号，自然流量号都有，咨询请加V:lijun1\n\n李军7：钱艳4：出售1万--100万粉丝 头条号，绑定头条号，然后同步抖音直接上粉，无上限，速度快，一次性搞定！可走淘宝担保交易！适合大批量上粉客户需求！赵勇6：lijun1\n\n李军7：承接 《小红书、视频号》--粉丝、评论、点赞、大拇指、评论点赞、小眼睛，看过等稳定上量，有速度，咨询请加V: lijun1 可对公，可发票！！\n\n李军7：Facebook/推特/ins/油管/tiktok/小红书 \n粉丝/点赞/评论/浏览量/转推等数据维护\n另出售各种白号、千粉、万粉账号！欢迎咨询：lijun2，可对公，可发票！\n\n李军7：专业提供TikToK全球满月白号、千粉号、5千粉号、万粉号，橱窗号等类型\n\n•美国？??????

2024-07-16 15:29:28 中国标准时间 - SparkPythonSDK - ERROR - [E:\python_env\iflytek\lib\site-packages\sparkai\llm\llm.py:689] - SparkLLMClient wait LLM api response timeout 60 seconds


'LLMResult' object has no attribute 'replace'
- - - - - - AttributeError- - - - - - - -
generations=[[ChatGeneration(text='提供！咨询请加V:lijun1\n\n李军7：承接视频号--粉丝、视频播放量、点赞、转发、收藏、评论、直播在线人气(看过)，直播互动、等业务！稳定上量，有速度，咨询请加V: lijun1\n\n李军7：专业提供各类社交平台账号（新号、老号、带粉号都有）：脸书、油管、推特、ins、TikTok(可指定国家和指定粉丝数量)、chatGPT、Reddit、领英等平台，保首登，现货可批量提供！咨询请加V:lijun1\n\n李军7：Facebook/推特/ins/油管/tiktok/小红书 \n粉丝/点赞/评论/浏览量/转推等数据维护\n另出售各种白号、千粉、万粉账号！欢迎咨询：lijun2，可对公，可发票！\n\n李军7：承接 《小红书、视频号》--粉丝、评论、点赞、大拇指、评论点赞、小眼睛，看过等稳定上量，有速度，咨询请加V: lijun1 可对公，可发票！！\n\n李军7：【拥有资源】源头一手Tik*Tok号商！美、英、马来、印尼、越、泰、新、菲等多个国家都有！\n【联系方式】加 郑娜6：lijun2\n\n李军7：TikTok全类型账号一手出售，满月白号，老号，满月千粉，英区，美区五千粉，还有印尼，马来，菲律宾，泰国，越南，沙特等全球满月千粉号，自然流量号都有，咨询请加V:lijun1\n\n李军7：钱艳4：出售1万--100万粉丝 头条号，绑定头条号，然后同步抖音直接上粉，无上限，速度快，一次性搞定！可走淘宝担保交易！适合大批量上粉客户需求！赵勇6：lijun1\n\n李军7：承接 《小红书、视频号》--粉丝、评论、点赞、大拇指、评论点赞、小眼睛，看过等稳定上量，有速度，咨询请加V: lijun1 可对公，可发票！！\n\n李军7：Facebook/推特/ins/油管/tiktok/小红书 \n粉丝/点赞/评论/浏览量/转推等数据维护\n另出售各种白号、千粉、万粉账号！欢迎咨询：lijun2，可对公，可发票！\n\n李军7：专业提供TikToK全球满月白号、千粉号、5千粉号、万粉号，橱窗号等类型\n\n•美国？??????

...

'LLMResult' object has no attribute 'replace'
- - - - - - AttributeError- - - - - - - -
generations=[[ChatGeneration(text='郑杰1询问赵强8是否在刊例价里看到了某些信息，赵强8回答没有。这段对话看起来像郑杰1表示那暂时就没有了。\n是一系列的工作交流，涉及多个\n赵强8提到商家转账到零钱被驳回，郑杰1建议查看详细提示信息。\n参与者之间的对话。主要内容包括：\n\n1.\n赵强8按照说明提交内容，但被驳回，他想知道模板需要怎么修改。\n\n郑杰1建议再提交一遍，如果还是不行，可以直接联系人工客服。\n\n赵强8请求发送Word文档以便修改，郑杰1表示只有PDF格式的文档。\n\n 郑杰1和赵强8讨论关于刊例价、商家转账到零钱被驳回的问题，以及如何修改模板重新提交。\n2.晚上好，郑杰1因工作岗位调整，接下来由周艳8老师提供服务对接。\n\n 郑杰1告知因工作岗位调整，接下来的服务将由周艳8老师提供服务对接。\n赵强8询问接下来如何配置，周艳8问需要什么内容，提到了分销手册。\n\n赵强8提到安全医生的文件需要部署，王艳9询问文件用途和如何协助。\n\n3. 赵强8询问关于配置、分销、安全医生的文件部署等问题，王艳9回应需要具体信息来协助。\n4.赵强8提到文档里有说明，并要求提供一个链接回来。\n\n周艳8表示会处理并重新上传链接。\n\n郑杰1分享了客户到店频次提升的案例。\n 周艳8提供关于分销手册的信息，并处理了一些技术问题，如链接添加、后台设置等。\n5.\n赵强8要求周艳8提供链接。\n\n周艳8指导赵强8在后台添加链接，并询问是否有某个选项。\n\n赵强8表示找到了填写IP的地方。\n\n周艳8确认是否可以使用，赵强8表示感谢。\n 郑杰1分享了关于企业微信和企业微信管家如何帮助客户增长的案例。\n6.\n周艳8播报了本周重点内容更新。\n\n 周艳8更新了本周的重点播报，涉及商城装修改造和新功能优化。\n7.郑杰1分享了使用企业微信提升新客增量和老客返店率的案例。\n\n赵强8询问如何让商城里的分销推广员失效，周艳8表示目前无法取消资格。\n\n郑杰1分享了学员、订单增长背后的案例。\n 赵强8询问如何让商城里的分销推广员失效，周艳

2024-07-16 18:22:55 中国标准时间 - SparkPythonSDK - ERROR - [E:\python_env\iflytek\lib\site-packages\sparkai\llm\llm.py:383] - No completion Generate


'LLMResult' object has no attribute 'replace'
- - - - - - AttributeError- - - - - - - -
generations=[[ChatGeneration(text=' 周强3询问关于百度竞价线索自动分配及部门设置的问题，王艳9确认系统会自动分配并根据设定轮流分配，同时指导如何在企业微信后台进行部门设置。\n这段对话记录涉及的主要内容包括3. 王磊1询问如何在企业微信后台同步通讯录部门设置到企微管家，王艳9提供了详细步骤。\n4.：\n\n 周强3请求开通会话存档功能试用，并询问是否能通过百度竞价账号的线索获得弹出提醒。\n5.1. 王磊1询问关于企业微信后台的部门设置问题，并被告知需要在企业微信后台进行设置。\n 郑杰1（巨石蓝海 企微管家）介入，提供会话存档试用的协助，并讨论了企业微信认证的必要性及相关操作。\n2. 周强3询问关于自动分配线索和会话存档功能试用的问题，得知需要开通会话存档功能。\n3. 郑杰1指导如何安装企微管家、认证企业微信以及配置会话存档功能。\n4.6. 讨论了关于企业微信认证的步骤，郑杰1指导小悦悦如何进行认证流程，并强调认证费用是支付给企业微信官方的。\n7. 提到了新的H5红包功能的上线，包括创建客户红包、指定金额或员工自定义金额的红包，以及防止红包被转发后不允许领取的功能。\n 最后，提到了企微管家正式版本的开通情况，并介绍了即将上线的新H5红包功能，包括支持创建客户红包、客户群红包，\n从这些信息中可以看出，这是一段关于企业微信管理工具使用和企业微信功能更新的讨论。可自定义金额等特性，并提示旧红包工具将在12月31日下线。\n\n其中涉及到了企业微信的部门设置、自动分配线索、会话存档功能试用、企微管家的安装与认证、以及新的H5红包功能的介绍。整体上，这是一段涉及企业微信管理、功能设置、新功能介绍及技术支持的对话记录。', message=AIMessage(content=' 周强3询问关于百度竞价线索自动分配及部门设置的问题，王艳9确认系统会自动分配并根据设定轮流分配，同时指导如何在企业微信后台进行部门设置。\n这段对话记录涉及的主要内容包括3. 王磊1询问如何在企业微信后台同步通讯录部门设置到企微管家，王艳9提供了详细步骤。\n4.：\n\n 周强

2024-07-16 18:25:05 中国标准时间 - SparkPythonSDK - ERROR - [E:\python_env\iflytek\lib\site-packages\sparkai\llm\llm.py:383] - No completion Generate


'LLMResult' object has no attribute 'replace'
- - - - - - AttributeError- - - - - - - -
generations=[[ChatGeneration(text='孙军2：\n孙军2：各位老师好，新的一年已经开启，我们诚邀您您好，这是一篇关于企业对我们2023年的服务和产品做出评价。辛苦在表格内反馈您的宝贵建议。微信的文章。您的反馈会帮助我们做的更好，感谢您的支持！文章中提到了一些常见问题的解决方法，例如如何查看版本、如何设置进群欢迎语等。此外，文章还提到了巨石蓝海如何帮助企业实现品牌新增长的案例。\n【收集表】 2023年度服务满意度评价', message=AIMessage(content='孙军2：\n孙军2：各位老师好，新的一年已经开启，我们诚邀您您好，这是一篇关于企业对我们2023年的服务和产品做出评价。辛苦在表格内反馈您的宝贵建议。微信的文章。您的反馈会帮助我们做的更好，感谢您的支持！文章中提到了一些常见问题的解决方法，例如如何查看版本、如何设置进群欢迎语等。此外，文章还提到了巨石蓝海如何帮助企业实现品牌新增长的案例。\n【收集表】 2023年度服务满意度评价'))]] llm_output={'token_usage': {'question_tokens': 3034, 'prompt_tokens': 3034, 'completion_tokens': 66, 'total_tokens': 3100}} run=[RunInfo(run_id=UUID('afda385e-d426-493f-afae-f922174c9b0a'))]
- - - - - - AttributeError- - - - - - - -
'LLMResult' object has no attribute 'replace'
- - - - - - AttributeError- - - - - - - -
generations=[[]] llm_output={} run=[RunInfo(run_id=UUID('6a7d769d-264f-42d6-8d44-994b4df79431'))]
- - - - - - AttributeError- - - - - - - 

In [25]:
train_data['chat_text'][0]

'李勇7：\n王艳9：您指的什么，辛苦截图告知下\n王艳9：您是企业微信管理员吗？\n王艳9：管理员可以看到\n王艳9：\n王艳9：\n王艳9：那让管理员给您添加应用负责人\n王艳9：也可以看到\n王艳9：@动漫周边配送中心\n王艳9：电脑端不可以\n王艳9：要登录管理端后台\n王艳9：这是一条引用/回复消息：\n"王艳9\n\n------\n这个网址\n王艳9：\n郑杰1：@动漫周边配送中心 您登记应用管理——点击打卡 里面有的哈 \n郑杰1：\n郑杰1：使用企业微信完成20%的新客增量提升，老客返店率增长至50%！\n巨石蓝海如何助力燕京啤酒旗下小酒馆实现品牌新增长？\n郑杰1：????上线功能：H5红包改版\n需求背景：当前的H5红包功能较弱，只支持创建一种金额的红包给员工发放，对需要给不同门店、不同业务线的员工创建不同的红包活动的场景，就不太好满足\n\n具体上新功能点介绍：\n①支持创建客户红包和客户群红包，员工在聊天工具栏可一键发送红包，用于1v1客户运营和客户群促活场景\n②可创建指定金额或员工自定义金额的红包，用于对客户的分层运营\n③支持转发红包给他人后不允许领取，防止被薅羊毛\n\n注：旧红包工具将在12月31日下线，请注意切换到新的红包功能使用 \n郑杰1：您是指要看到什么内容呢 \n郑杰1：看不到这个新客户页面是吗 \n郑杰1：\n郑杰1：您的企业微信更新到最新版本之后 退出 重新登录再试试呢 \n郑杰1：电脑端的话 你点击你的头像 点击设置 点击关于 \n郑杰1：检查版本 更新\n郑杰1：\n郑杰1：你点击关于里面\n郑杰1：去更新版本之后再看看 \n周杰5：学员、订单双增长的背后，EF是如何用好企业微信和企微管家优化服务链路，让“短期教学”变成“终身教育”？\n周杰5：\n周杰5：各位老师好，新的一年已经开启，我们诚邀您对我们2023年的服务和产品做出评价。辛苦在表格内反馈您的宝贵建议，您的反馈会帮助我们做的更好，感谢您的支持！\n【收集表】 2023年度服务满意度评价\xa0 \n王艳9：\n王艳9：这里看下呢\n王艳9：点头像\n王艳9：然后最下面有个设置\n王艳9：重启试下把'

In [26]:
train_data['new_chat_text'][0]

'李勇7：\n王艳9：您指的什么，辛苦截图告知下\n王艳9：您是企业微信管理员吗？\n王艳9：管理员可以看到\n王艳9：\n王艳9：\n王艳9：那让管理员给您添加应用负责人\n王艳9：也可以看到\n王艳9：@动漫周边配送中心\n王艳9：电脑端不可以\n王艳9：要登录管理端后台\n王艳9：这是一条引用/回复消息：\n"王艳9\n\n------\n这个网址\n王艳9：\n郑杰1：@动漫周边配送中心 您登记应用管理——点击打卡 里面有的哈 \n郑杰1：\n郑杰1：使用企业微信完成20%的新客增量提升，老客返店率增长至50%！\n巨石蓝海如何助力燕京啤酒旗下小酒馆实现品牌新增长？\n郑杰1：????上线功能：H5红包改版\n需求背景：当前的H5红包功能较弱，只支持创建一种金额的红包给员工发放，对需要给不同门店、不同业务线的员工创建不同的红包活动的场景，就不太好满足\n\n具体上新功能点介绍：\n①支持创建客户红包和客户群红包，员工在聊天工具栏可一键发送红包，用于1v1客户运营和客户群促活场景\n②可创建指定金额或员工自定义金额的红包，用于对客户的分层运营\n③支持转发红包给他人后不允许领取，防止被薅羊毛\n\n注：旧红包工具将在12月31日下线，请注意切换到新的红包功能使用 \n郑杰1：您是指要看到什么内容呢 \n郑杰1：看不到这个新客户页面是吗 \n郑杰1：\n郑杰1：您的企业微信更新到最新版本之后 退出 重新登录再试试呢 \n郑杰1：电脑端的话 你点击你的头像 点击设置 点击关于 \n郑杰1：检查版本 更新\n郑杰1：\n郑杰1：你点击关于里面\n郑杰1：去更新版本之后再看看 \n周杰5：学员、订单双增长的背后，EF是如何用好企业微信和企微管家优化服务链路，让“短期教学”变成“终身教育”？\n周杰5：\n周杰5：各位老师好，新的一年已经开启，我们诚邀您对我们2023年的服务和产品做出评价。辛苦在表格内反馈您的宝贵建议，您的反馈会帮助我们做的更好，感谢您的支持！\n【收集表】 2023年度服务满意度评价\xa0 \n王艳9：\n王艳9：这里看下呢\n王艳9：点头像\n王艳9：然后最下面有个设置\n王艳9：重启试下把'

In [ ]:
al_infos_0":[{"基本信息-姓名":"钱艳4",
"基本信息-手机号码":"16795386856",
"基本信息-邮箱":"","基本信息-地区":"",
"基本信息-详细地址":"",
"基本信息-性别":"",
"基本信息-年龄":"",
"基本信息-生日":"",
"咨询类型":["答疑"],
"意向产品":[],
"购买异议点":["产品功能"],
"客户预算-预算是否充足":"",
"客户预算-总体预算金额":"",
"客户预算-预算明细":"",
"竞品信息":"",
"客户是否有意向":"",
"客户是否有卡点":"有卡点",
"客户购买阶段":"增购",
"下一步跟进计划-参与人":[]
,"下一步跟进计划-时间点":"",
"下一步跟进计划-具体事项":""}]}